In [11]:
from engine import trainer
from utils import plot_results, LinearLR, WarmupLR
from models.model import get_model
from torch.utils.data import ConcatDataset 
from dataset import CUBDataset, FGVCAircraft, FOODDataset

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import v2
import torchvision
from torch.utils.data import default_collate
torchvision.disable_beta_transforms_warning()
from engine import val_step


import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd

import yaml
import json
import time
import os


In [12]:
config = yaml.load(open('finetune_config.yaml', 'r'), Loader=yaml.FullLoader)


LEARNING_RATE = [float(i) for i in config["LEARNING_RATE"]]
LEARNING_SCHEDULER = config["LEARNING_SCHEDULER"]
BATCH_SIZE = int(config["BATCH_SIZE"])
NUM_EPOCHS = int(config["NUM_EPOCHS"])

LOSS = config["LOSS"]
LABEL_SMOOTHING = float(config["LABEL_SMOOTHING"])

IMAGE_SIZE = int(config["IMAGE_SIZE"])
MODEL = config["MODEL"]
INIT_PATH = config["INIT_PATH"]
PRETRAINED = config["PRETRAINED"]
FREEZE = config["FREEZE"]

DATASET = config["DATASET"]
CUT_UP_MIX = config["CUT_UP_MIX"]
HPC = config["HPC"]

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {DEVICE} device")

Using cuda device


In [13]:
state_dicts = []
val_results = []
WEIGHT_PATH = "/home/nuren.zhaksylyk/Documents/CV703/CV703_Assignment_1/runs/finetune/2024-01-27_20-46-18/CUB/ConvTransNeXtTiny"

def START_seed():
    seed = 9
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed) 

In [14]:
START_seed()

dataset_name = ["CUB", "CUB and FGVC-Aircraft", "FoodX"][DATASET]
num_classes = [200, 200 + 100, 251][DATASET]

#run id is date and time of the run

save_root_dir = "./runs/finetune/" + dataset_name + '/' + MODEL
os.makedirs(save_root_dir, exist_ok=True)



#load data
transforms_train = v2.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop((IMAGE_SIZE, IMAGE_SIZE), scale=(0.7, 1.0), antialias=True),

    # v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET, ),
    v2.RandAugment(num_ops=2, magnitude=10),
    v2.RandomErasing(p=0.1),

    v2.ToDtype(torch.float, scale=True),
    v2.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
])

transforms_test = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((224, 224), antialias=True),
    v2.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
])


cutmix = v2.CutMix(num_classes=num_classes, alpha=1.0)
mixup = v2.MixUp(num_classes=num_classes, alpha=0.2)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

if CUT_UP_MIX:
    def collate_fn(batch):
        return cutmix_or_mixup(*default_collate(batch))
else:
    collate_fn = None

if HPC:
    dataset_path_prefix = "/apps/local/shared/CV703/datasets/"
else:
    dataset_path_prefix = "datasets/"

if dataset_name == 'CUB':
    dataset_path = dataset_path_prefix + "CUB/CUB_200_2011"

    train_simple_dataset = CUBDataset(image_root_path=dataset_path, transform=transforms_test, split="train")
    train_dataset = CUBDataset(image_root_path=dataset_path, transform=transforms_train, split="train")
    test_dataset = CUBDataset(image_root_path=dataset_path, transform=transforms_test, split="test")

    train_simple_loader = DataLoader(train_simple_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

    class_names = train_dataset.classes

elif dataset_name == 'CUB and FGVC-Aircraft':

    dataset_path_cub = dataset_path_prefix + "CUB/CUB_200_2011"
    train_simple_dataset_cub = CUBDataset(image_root_path=dataset_path_cub, transform=transforms_test, split="train")
    train_dataset_cub = CUBDataset(image_root_path=dataset_path_cub, transform=transforms_train, split="train")
    test_dataset_cub = CUBDataset(image_root_path=dataset_path_cub, transform=transforms_test, split="test")

    dataset_path_aircraft = dataset_path_prefix + "fgvc-aircraft-2013b"
    train_simple_dataset_aircraft = FGVCAircraft(root=dataset_path_aircraft, transform=transforms_test, train=True)
    train_dataset_aircraft = FGVCAircraft(root=dataset_path_aircraft, transform=transforms_train, train=True)
    test_dataset_aircraft = FGVCAircraft(root=dataset_path_aircraft, transform=transforms_test, train=False)

    concat_dataset_train_simple = ConcatDataset([train_simple_dataset_cub, train_simple_dataset_aircraft])
    concat_dataset_train = ConcatDataset([train_dataset_cub, train_dataset_aircraft])
    concat_dataset_test = ConcatDataset([test_dataset_cub, test_dataset_aircraft])

    train_simple_loader = torch.utils.data.DataLoader(
                concat_dataset_train_simple,
                batch_size=BATCH_SIZE, shuffle=True,
                num_workers=8, pin_memory=True,
                collate_fn=None
                )
    train_loader = torch.utils.data.DataLoader(
                concat_dataset_train,
                batch_size=BATCH_SIZE, shuffle=True,
                num_workers=8, pin_memory=True,
                collate_fn=collate_fn
                )
    test_loader = torch.utils.data.DataLoader(
                concat_dataset_test,
                batch_size=BATCH_SIZE, shuffle=False,
                num_workers=8, pin_memory=True
                )
    
    classes_1 = concat_dataset_train.datasets[0].classes
    classes_2 = concat_dataset_train.datasets[1].classes

    class_names = [*classes_1, *classes_2]

elif dataset_name == 'FoodX':
    dataset_path = dataset_path_prefix + "FoodX/food_dataset"

    train_dataset = FOODDataset(data_dir=dataset_path, transform=transforms_train, split="train")
    test_dataset = FOODDataset(data_dir=dataset_path, transform=transforms_test, split="val")

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)






In [15]:
state_dicts = []
val_results = []
test_results = []

#if there is no test directory create it
if not os.path.exists("test"):
    os.makedirs("test")

#if there is no folder named after dataset inside test directory create it
if not os.path.exists(os.path.join("test", dataset_name, MODEL)):
    os.makedirs(os.path.join("test", dataset_name, MODEL))

test_save_path = os.path.join("test", dataset_name, MODEL)

In [16]:
def souping(model, state_dicts, alphal):
    sd = {k : state_dicts[0][k].clone() * alphal[0] for k in state_dicts[0].keys()}
    for i in range(1, len(state_dicts)):
        for k in state_dicts[i].keys():
            sd[k] = sd[k] + state_dicts[i][k].clone() * alphal[i]
    model.load_state_dict(sd)
    return model

In [25]:
def greedy_souping(state_dicts, val_results, val_loader, loss, DEVICE, num_classes):
    ranked_candidates = [i for i in range(len(state_dicts))]
    ranked_candidates.sort(key=lambda x: -val_results[x])
    print(ranked_candidates)
    current_best = val_results[ranked_candidates[0]]
    print('currentttttttt bestttttttt', current_best)
    best_ingredients = ranked_candidates[:1]

    for i in range(1, len(state_dicts)):
        # add current index to the ingredients
        ingredient_indices = best_ingredients + [ranked_candidates[i]]
        alphal = [0 for i in range(len(state_dicts))]
        for j in ingredient_indices:
            alphal[j] = 1 / len(ingredient_indices)

        # benchmark and conditionally append
        model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

        greedy_model = souping(model, state_dicts, alphal)
        greedy_model.to(DEVICE)
        greedy_val_loss, greedy_val_acc = val_step(greedy_model, val_loader, loss, DEVICE)
        print(f'Models {ingredient_indices} got {greedy_val_acc} on validation.')
        if greedy_val_acc > current_best:
            current_best = greedy_val_acc
            best_ingredients = ingredient_indices
       

    alphal = [0 for i in range(len(state_dicts))]
    for j in best_ingredients:
        alphal[j] = 1 / len(best_ingredients)
    greedy_model = souping(model, state_dicts, alphal)
    return greedy_model, best_ingredients


In [26]:
def greedy_souping_2(state_dicts, val_results, val_loader, loss, DEVICE, num_classes):
    ranked_candidates = [i for i in range(len(state_dicts))]
    ranked_candidates.sort(key=lambda x: -val_results[x])
    print(ranked_candidates)
    current_best = val_results[ranked_candidates[0]]
    print('currentttttttt bestttttttt', current_best)
    best_ingredients = ranked_candidates[:1]
    should_restart = True
    while should_restart:
        should_restart = False
        for i in range(1, len(state_dicts)):
            if ranked_candidates[i] in best_ingredients:
                continue
            # add current index to the ingredients
            ingredient_indices = best_ingredients + [ranked_candidates[i]]
            alphal = [0 for i in range(len(state_dicts))]
            for j in ingredient_indices:
                alphal[j] = 1 / len(ingredient_indices)

            # benchmark and conditionally append
            model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

            greedy_model = souping(model, state_dicts, alphal)
            greedy_model.to(DEVICE)
            greedy_val_loss, greedy_val_acc = val_step(greedy_model, val_loader, loss, DEVICE)
            print(f'Models {ingredient_indices} got {greedy_val_acc} on validation.')
            if greedy_val_acc > current_best:
                should_restart = True
                current_best = greedy_val_acc
                best_ingredients = ingredient_indices
       

    alphal = [0 for i in range(len(state_dicts))]
    for j in best_ingredients:
        alphal[j] = 1 / len(best_ingredients)
    greedy_model = souping(model, state_dicts, alphal)
    return greedy_model, best_ingredients


In [29]:
import random
                                        
def prune_souping_random(state_dicts, val_results, val_loader, loss, DEVICE, num_classes):
    ranked_candidates = [i for i in range(len(state_dicts))]
    ranked_candidates.sort(key=lambda x: -val_results[x])
    print(ranked_candidates)
    current_best = val_results[ranked_candidates[0]]
    print('currentttttttt bestttttttt', current_best)
    best_ingredients = ranked_candidates
    i_s = []
    while True:
        if len(best_ingredients) == 0:
            break
        
        b = [x for x in range(len(best_ingredients)) if x not in i_s]
        
        if len(b) == 0:
            break

        i = random.choice(b)
        i_s.append(i)
        
        ingredient_indices = best_ingredients[:i] + best_ingredients[i+1:]
        alphal = [0 for i in range(len(state_dicts))]
        for j in ingredient_indices:
            alphal[j] = 1 / len(ingredient_indices)
        model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

        greedy_model = souping(model, state_dicts, alphal)
        greedy_model.to(DEVICE)
        greedy_val_loss, greedy_val_acc = val_step(greedy_model, val_loader, loss, DEVICE)
        print(f'Models {ingredient_indices} got {greedy_val_acc} on validation.')
        if greedy_val_acc > current_best:
            current_best = greedy_val_acc
            best_ingredients = ingredient_indices
            i_s = []
            # benchmark and conditionally append
            
       

    alphal = [0 for i in range(len(state_dicts))]
    for j in best_ingredients:
        alphal[j] = 1 / len(best_ingredients)
    greedy_model = souping(model, state_dicts, alphal)
    return greedy_model, best_ingredients


In [30]:
                                 
def prune_souping_sorted(state_dicts, val_results, val_loader, loss, DEVICE, num_classes):
    ranked_candidates = [i for i in range(len(state_dicts))]
    ranked_candidates.sort(key=lambda x: -val_results[x])
    print(ranked_candidates)
    current_best = val_results[ranked_candidates[0]]
    print('currentttttttt bestttttttt', current_best)
    best_ingredients = ranked_candidates
    i = 0
    while True:
        if len(best_ingredients) == 0:
            break

        if i >= len(best_ingredients):
            break
        
        ingredient_indices = best_ingredients[:i] + best_ingredients[i+1:]
        alphal = [0 for i in range(len(state_dicts))]
        for j in ingredient_indices:
            alphal[j] = 1 / len(ingredient_indices)
        model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

        greedy_model = souping(model, state_dicts, alphal)
        greedy_model.to(DEVICE)
        greedy_val_loss, greedy_val_acc = val_step(greedy_model, val_loader, loss, DEVICE)
        print(f'Models {ingredient_indices} got {greedy_val_acc} on validation.')
        if greedy_val_acc > current_best:
            current_best = greedy_val_acc
            best_ingredients = ingredient_indices
            i = 0
            # benchmark and conditionally append
        i+=1
       

    alphal = [0 for i in range(len(state_dicts))]
    for j in best_ingredients:
        alphal[j] = 1 / len(best_ingredients)
    greedy_model = souping(model, state_dicts, alphal)
    return greedy_model, best_ingredients

In [19]:
for idx, folder in enumerate(sorted(os.listdir(WEIGHT_PATH))):

    model_path = os.path.join(WEIGHT_PATH, folder)
    #read config in train_summary.json
    train_summary = json.load(open(os.path.join(model_path, "train_summary.json"), 'r'))
    model_config = train_summary["config"]
    
    #load model
    checkpoint = torch.load(os.path.join(model_path, "best_model.pth"), map_location=DEVICE)
    state_dicts.append(checkpoint)
    model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
    model.load_state_dict(checkpoint)
    model = model.to(DEVICE)
    #load loss function
    LOSS = model_config["LOSS"]
    if LOSS == "MSE":
        loss = torch.nn.MSELoss()
    elif LOSS == "L1Loss":
        loss = torch.nn.L1Loss()
    elif LOSS == "SmoothL1Loss":
        loss = torch.nn.SmoothL1Loss()
    elif LOSS == "CrossEntropyLoss":
        loss = torch.nn.CrossEntropyLoss()
    elif LOSS == "BCEWithLogitsLoss":
        loss = torch.nn.BCEWithLogitsLoss()
    
    val_loss, val_acc = val_step(model, test_loader, loss, device = DEVICE)

    val_results.append({'Model Name': idx,
                                    'Val Accuracy': val_acc,
                                    'LR': train_summary["results"]["learning_rate"][0],
                                    })





100%|##########| 182/182 [00:16<00:00, 10.91it/s]


In [23]:
results_val_df = pd.DataFrame(val_results)

val_copy = results_val_df.copy()
sorted_val = val_copy.sort_values(by= 'Val Accuracy',ascending=False)
sorted_val.to_csv(os.path.join(test_save_path, "VAL_RESULTS.csv"), index=False)

print(f"Best model Val Accuracy: {sorted_val.iloc[0]['Val Accuracy']}")
print(f"Second best model Val Accuracy: {sorted_val.iloc[1]['Val Accuracy']}")
print(f"Worst model Val Accuracy: {sorted_val.iloc[-1]['Val Accuracy']}")

#UNIFORM
print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to(DEVICE)

uniform_test_loss, uniform_test_acc  = val_step(uniform_model, test_loader, loss, DEVICE)
print(f"Uniform souping accuracy: {uniform_test_acc}")

#greedy

print("Greedy souping ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping_2(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")



Best model Val Accuracy: 0.8626164998274076
Second best model Val Accuracy: 0.8603727994477045
Worst model Val Accuracy: 0.8117017604418364
Uniform souping ...


/home/nuren.zhaksylyk/.conda/envs/CV701_env/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|##########| 182/182 [00:18<00:00,  9.84it/s]


Uniform souping accuracy: 0.8629616845012081
Greedy souping ...
[4, 6, 0, 5, 10, 11, 9, 8, 7, 1, 2, 3]
currentttttttt bestttttttt 0.8626164998274076


100%|##########| 182/182 [00:22<00:00,  8.26it/s]


Models [4, 6] got 0.8631342768381084 on validation.


...

Models [4, 6, 10, 1, 11, 0, 2] got 0.8653779772178115 on validation.


100%|##########| 182/182 [00:16<00:00, 11.22it/s]


Models [4, 6, 10, 1, 11, 0, 3] got 0.8636520538488092 on validation.
VAL INGREDIENTS [4, 6, 10, 1, 11, 0]


100%|##########| 182/182 [00:20<00:00,  9.02it/s]

Greedy souping accuracy: 0.8669313082499137


In [27]:
results_val_df = pd.DataFrame(val_results)

val_copy = results_val_df.copy()
sorted_val = val_copy.sort_values(by= 'Val Accuracy',ascending=False)
sorted_val.to_csv(os.path.join(test_save_path, "VAL_RESULTS.csv"), index=False)

print(f"Best model Val Accuracy: {sorted_val.iloc[0]['Val Accuracy']}")
print(f"Second best model Val Accuracy: {sorted_val.iloc[1]['Val Accuracy']}")
print(f"Worst model Val Accuracy: {sorted_val.iloc[-1]['Val Accuracy']}")

#UNIFORM
print("Uniform souping ...")
alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
uniform_model = souping(model, state_dicts, alphal)
uniform_model.to(DEVICE)

uniform_test_loss, uniform_test_acc  = val_step(uniform_model, test_loader, loss, DEVICE)
print(f"Uniform souping accuracy: {uniform_test_acc}")

#greedy

print("Greedy souping ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = greedy_souping(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")



Best model Val Accuracy: 0.8626164998274076
Second best model Val Accuracy: 0.8603727994477045
Worst model Val Accuracy: 0.8117017604418364
Uniform souping ...


100%|##########| 182/182 [00:20<00:00,  8.71it/s]


Uniform souping accuracy: 0.8629616845012081
Greedy souping ...
[4, 6, 0, 5, 10, 11, 9, 8, 7, 1, 2, 3]
currentttttttt bestttttttt 0.8626164998274076


100%|##########| 182/182 [00:17<00:00, 10.31it/s]


Models [4, 6] got 0.8631342768381084 on validation.


...

Models [4, 6, 10, 1, 2] got 0.8629616845012081 on validation.


100%|##########| 182/182 [00:17<00:00, 10.40it/s]


Models [4, 6, 10, 1, 3] got 0.8626164998274076 on validation.
VAL INGREDIENTS [4, 6, 10, 1]


100%|##########| 182/182 [00:21<00:00,  8.66it/s]

Greedy souping accuracy: 0.8658957542285123


In [28]:
results_val_df = pd.DataFrame(val_results)

val_copy = results_val_df.copy()
sorted_val = val_copy.sort_values(by= 'Val Accuracy',ascending=False)
sorted_val.to_csv(os.path.join(test_save_path, "VAL_RESULTS.csv"), index=False)

print(f"Best model Val Accuracy: {sorted_val.iloc[0]['Val Accuracy']}")
print(f"Second best model Val Accuracy: {sorted_val.iloc[1]['Val Accuracy']}")
print(f"Worst model Val Accuracy: {sorted_val.iloc[-1]['Val Accuracy']}")

#UNIFORM
# print("Uniform souping ...")
# alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
# model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
# uniform_model = souping(model, state_dicts, alphal)
# uniform_model.to(DEVICE)

# uniform_test_loss, uniform_test_acc  = val_step(uniform_model, test_loader, loss, DEVICE)
# print(f"Uniform souping accuracy: {uniform_test_acc}")

#greedy

print("Greedy souping ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = prune_souping_random(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")



Best model Val Accuracy: 0.8626164998274076
Second best model Val Accuracy: 0.8603727994477045
Worst model Val Accuracy: 0.8117017604418364
Uniform souping ...


100%|##########| 182/182 [00:18<00:00,  9.91it/s]


Uniform souping accuracy: 0.8629616845012081
Greedy souping ...
[4, 6, 0, 5, 10, 11, 9, 8, 7, 1, 2, 3]
currentttttttt bestttttttt 0.8626164998274076


100%|##########| 182/182 [00:20<00:00,  8.95it/s]


Models [4, 6, 5, 10, 11, 9, 8, 7, 1, 2, 3] got 0.8631342768381084 on validation.


...

Models [6, 5, 8, 7, 1, 2, 3] got 0.8631342768381084 on validation.


100%|##########| 182/182 [00:18<00:00,  9.65it/s]


Models [6, 5, 11, 8, 7, 2, 3] got 0.8624439074905074 on validation.
VAL INGREDIENTS [6, 5, 11, 8, 7, 1, 2, 3]


100%|##########| 182/182 [00:18<00:00, 10.10it/s]

Greedy souping accuracy: 0.8641698308595098


In [31]:
results_val_df = pd.DataFrame(val_results)

val_copy = results_val_df.copy()
sorted_val = val_copy.sort_values(by= 'Val Accuracy',ascending=False)
sorted_val.to_csv(os.path.join(test_save_path, "VAL_RESULTS.csv"), index=False)

print(f"Best model Val Accuracy: {sorted_val.iloc[0]['Val Accuracy']}")
print(f"Second best model Val Accuracy: {sorted_val.iloc[1]['Val Accuracy']}")
print(f"Worst model Val Accuracy: {sorted_val.iloc[-1]['Val Accuracy']}")

#UNIFORM
# print("Uniform souping ...")
# alphal = [1 / len(state_dicts) for i in range(len(state_dicts))]
# model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)
# uniform_model = souping(model, state_dicts, alphal)
# uniform_model.to(DEVICE)

# uniform_test_loss, uniform_test_acc  = val_step(uniform_model, test_loader, loss, DEVICE)
# print(f"Uniform souping accuracy: {uniform_test_acc}")

#greedy

print("Greedy souping ...")
model = get_model(MODEL, PRETRAINED, num_classes, FREEZE)

val_res = list(results_val_df['Val Accuracy'])
greedy_model, best_ingredients = prune_souping_sorted(state_dicts, val_res, test_loader, loss, DEVICE, num_classes)
greedy_model.to(DEVICE)
print('VAL INGREDIENTS',best_ingredients)
greedy_test_loss, greedy_test_acc = val_step(greedy_model, test_loader, loss, DEVICE)

print(f"Greedy souping accuracy: {greedy_test_acc}")



Best model Val Accuracy: 0.8626164998274076
Second best model Val Accuracy: 0.8603727994477045
Worst model Val Accuracy: 0.8117017604418364
Uniform souping ...


100%|##########| 182/182 [00:19<00:00,  9.28it/s]


Uniform souping accuracy: 0.8629616845012081
Greedy souping ...
[4, 6, 0, 5, 10, 11, 9, 8, 7, 1, 2, 3]
currentttttttt bestttttttt 0.8626164998274076


100%|##########| 182/182 [00:19<00:00,  9.41it/s]


Models [6, 0, 5, 10, 11, 9, 8, 7, 1, 2, 3] got 0.8627890921643079 on validation.


...

100%|##########| 182/182 [00:18<00:00,  9.89it/s]


Models [6, 11, 9, 8, 7, 1, 2, 3] got 0.8638246461857093 on validation.


100%|##########| 182/182 [00:19<00:00,  9.31it/s]


Models [6, 10, 9, 8, 7, 1, 2, 3] got 0.8639972385226096 on validation.


 60%|#####9    | 109/182 [00:09<00:05, 13.32it/s]

In [ ]:
# #saving results in table
# print("Creating table ...")
# table = pd.DataFrame(columns=['Model Name', 'Test Accuracy', 'Test F1', 'Test Recall', 'Test Kappa','Test AUC','Augmentation', 'Learning Rate', 'SEED'])

# #get best, second best and worst model from sorted_test and add them to the table
# best_model = sorted_test.iloc[0]
# second_best_model = sorted_test.iloc[1]
# worst_model = sorted_test.iloc[-1]

# #rename model name in best, second best and worst model
# best_model['Model Name'] = f"Best 1: {best_model['Model Name']}"
# second_best_model['Model Name'] = f"Best 2: {second_best_model['Model Name']}"
# worst_model['Model Name'] = f"Worst: {worst_model['Model Name']}"

# table.loc[0] = best_model
# table.loc[1] = second_best_model
# table.loc[2] = worst_model

# #add uniform and greedy to the table
# table.loc[3] = {'Model Name': 'Uniform',
#                                     'Test Accuracy': uniform_test_acc,
#                                     }

# table.loc[4] = {'Model Name': 'Greedy',
#                                     'Test Accuracy': greedy_test_acc,
#                                     }



# #save the table to csv without index
# table.to_csv(os.path.join(test_save_path, "ALL.csv"), index=False)
# print(table.to_markdown())
# print("Table saved to ALL.csv")


# lr_state_dicts = state_dicts.copy()
# val_results_copy = results_val_df.copy()




# for j in range(3,8):
#     print(f">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>LR{j}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
#     indexes = []
#     for i in val_results_copy.iterrows():
#         if f"-0{j}" in i[1]['Learning Rate']:
#             indexes.append(i[0])
    
#     #remove from val_results_copy all the rows with indexes and reset the index
#     val_results_copy = val_results_copy.drop(indexes).reset_index(drop=True)
#     test_results_copy = test_results_copy.drop(indexes).reset_index(drop=True)

#     sorted_val = val_results_copy.sort_values(by= val_sort_by,ascending=False)
#     sorted_test = test_results_copy.sort_values(by= test_sort_by,ascending=False)

#     sorted_val.to_csv(os.path.join(test_save_path, f"VAL_RESULTS_LR<-0{j}.csv"), index=False)
#     sorted_test.to_csv(os.path.join(test_save_path, f"TEST_RESULTS_LR<-0{j}.csv"), index=False)

#     #remove from lr_state_dicts all the state_dicts with indexes
#     lr_state_dicts = [i for j, i in enumerate(lr_state_dicts) if j not in indexes]

#     #if state_dict is empty break
#     if len(lr_state_dicts) == 0:
#         print("No more models to analyze!")
#         break

#     print(f"Models with Learning rate -0{j} removed ...")

#     print(f"Best model {val_sort_by}: {sorted_val.iloc[0][val_sort_by]}")
#     print(f"Second best model {val_sort_by}: {sorted_val.iloc[1][val_sort_by]}")
#     print(f"Worst model {val_sort_by}: {sorted_val.iloc[-1][val_sort_by]}")
#     #UNIFORM
#     print("Unifrom souping ...")
#     alphal = [1 / len(lr_state_dicts) for i in range(len(lr_state_dicts))]
#     model = get_model(model_config["MODEL"], num_classes=NUM_CLASSES)
#     uniform_model = souping(model, lr_state_dicts, alphal)
#     uniform_model.to(DEVICE)

#     uniform_test_loss, uniform_test_acc, uniform_test_f1, uniform_test_recall, uniform_test_kappa, uniform_test_auc = val_step(uniform_model, test_loader, train_loader, loss, DEVICE, CLASSIFICATION)

#     #greedy
#     print("Greedy souping ...")
#     model = get_model(model_config["MODEL"], num_classes=NUM_CLASSES)

#     val_metric = list(val_results_copy[val_sort_by])
#     greedy_model, best_ingredients = greedy_souping(lr_state_dicts, val_metric, model_config["MODEL"], NUM_CLASSES, val_loader, train_loader, loss, DEVICE, CLASSIFICATION, val_sort_by)
#     greedy_model.to(DEVICE)
#     print(best_ingredients)
#     greedy_test_loss, greedy_test_acc, greedy_test_f1, greedy_test_recall, greedy_test_kappa, greedy_test_auc = val_step(greedy_model, test_loader, train_loader, loss, DEVICE, CLASSIFICATION)

#     test_metric = list(results_test_df[test_sort_by])
#     greedy_model_test, best_ingredients_test = greedy_souping(state_dicts, test_metric, model_config["MODEL"], NUM_CLASSES, test_loader, train_loader, loss, DEVICE, CLASSIFICATION, test_sort_by)
#     print(best_ingredients_test)
#     greedy_model_test.to(DEVICE)
#     greedy_test_loss_test, greedy_test_acc_test, greedy_test_f1_test, greedy_test_recall_test, greedy_test_kappa_test, greedy_test_auc_test = val_step(greedy_model_test, test_loader, train_loader, loss, DEVICE, CLASSIFICATION)



#     #saving results in table
#     print("Creating table ...")
#     table = pd.DataFrame(columns=['Model Name', 'Test Accuracy', 'Test F1', 'Test Recall','Test kAPPA','Test AUC', 'Augmentation', 'Learning Rate', 'SEED'])

#     #get best, second best and worst model from sorted_test and add them to the table
#     best_model = sorted_test.iloc[0]
#     second_best_model = sorted_test.iloc[1]
#     worst_model = sorted_test.iloc[-1]

#     #rename model name in best, second best and worst model
#     best_model['Model Name'] = f"Best 1: {best_model['Model Name']}"
#     second_best_model['Model Name'] = f"Best 2: {second_best_model['Model Name']}"
#     worst_model['Model Name'] = f"Worst: {worst_model['Model Name']}"

#     table.loc[0] = best_model
#     table.loc[1] = second_best_model
#     table.loc[2] = worst_model

#     #add uniform and greedy to the table
#     table.loc[3] = {'Model Name': 'Uniform',
#                                         'Test Accuracy': uniform_test_acc,
#                                         'Test F1': uniform_test_f1,
#                                         'Test Recall': uniform_test_recall,
#                                         'Test Kappa': uniform_test_kappa,
#                                         'Test AUC': uniform_test_auc,
#                                         'Augmentation': 'None',
#                                         'Learning Rate': 'None',
#                                         'SEED': 'None'}

#     table.loc[4] = {'Model Name': 'Greedy',
#                                         'Test Accuracy': greedy_test_acc,
#                                         'Test F1': greedy_test_f1,
#                                         'Test Recall': greedy_test_recall,
#                                         'Test Kappa': greedy_test_kappa,
#                                         'Test AUC': greedy_test_auc,
#                                         'Augmentation': 'None',
#                                         'Learning Rate': 'None',
#                                         'SEED': 'None'}

#     #save the table to csv without index
#     table.to_csv(os.path.join(test_save_path, f"LR<-0{j}.csv"), index=False)
#     print(table.to_markdown())
#     print(f"Table saved to LR<-0{j}.csv")